In [3]:
%pip install pandas

     ---------------------------------------- 0.0/60.8 kB ? eta -:--:--
     -------------------------- ------------- 41.0/60.8 kB 2.0 MB/s eta 0:00:01
     ---------------------------------------- 60.8/60.8 kB 1.1 MB/s eta 0:00:00
   ---------------------------------------- 0.0/11.6 MB ? eta -:--:--
   - -------------------------------------- 0.4/11.6 MB 11.6 MB/s eta 0:00:01
   --- ------------------------------------ 1.1/11.6 MB 13.4 MB/s eta 0:00:01
   ------- -------------------------------- 2.3/11.6 MB 18.0 MB/s eta 0:00:01
   -------------- ------------------------- 4.3/11.6 MB 24.9 MB/s eta 0:00:01
   ------------------------- -------------- 7.5/11.6 MB 34.0 MB/s eta 0:00:01
   -------------------------------------- - 11.3/11.6 MB 54.4 MB/s eta 0:00:01
   ---------------------------------------- 11.6/11.6 MB 50.4 MB/s eta 0:00:00
   ---------------------------------------- 0.0/12.9 MB ? eta -:--:--
   --------------- ------------------------ 4.9/12.9 MB 106.2 MB/s eta 0:00:01
 


[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: C:\Users\mannat2\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


# Goal: Build a classification model to segment the customers based on the account data.

classify the Accounts eligible for Credit Line Increase without Risk, Accounts eligible for Credit Line
Increase but has risk of potential defaults or fraud, No Credit Line Increase required and Non-Performing
accounts that pose a high risk.

# Features to look into
rams_batch_cur['cu_bhv_scr'] - internal old behavior score which helps credit card issuers with a scoring system, higher is better

statement_fact['return_check_cnt_total'] - total number of returned checks for the account

In [ ]:
import pandas as pd

In [24]:
account_dim = pd.read_csv("C:/Users/mannat2/datathon/consumer_spending_ml/data/tmp/datathon/data/account_dim_20250325.csv")
fraud_claim_case = pd.read_csv("C:/Users/mannat2/datathon/consumer_spending_ml/data/tmp/datathon/data/fraud_claim_case_20250325.csv")
fraud_claim_tran = pd.read_csv("C:/Users/mannat2/datathon/consumer_spending_ml/data/tmp/datathon/data/fraud_claim_tran_20250325.csv")
rams_batch_cur = pd.read_csv("C:/Users/mannat2/Documents/GitHub/consumer_spending_ml/data/tmp/datathon/data/rams_batch_cur_20250325.csv")
statement_fact = pd.read_csv("C:/Users/mannat2/Documents/GitHub/consumer_spending_ml/data/tmp/datathon/data/statement_fact_20250325.csv")
syf_id = pd.read_csv("C:/Users/mannat2/Documents/GitHub/consumer_spending_ml/data/tmp/datathon/data/syf_id_20250325.csv")
transfaction_fact = pd.read_csv("C:/Users/mannat2/Documents/GitHub/consumer_spending_ml/data/tmp/datathon/data/transaction_fact_20250325.csv")
wrld_stor_tran_fact = pd.read_csv("C:/Users/mannat2/Documents/GitHub/consumer_spending_ml/data/tmp/datathon/data/wrld_stor_tran_fact_20250325.csv")

In [30]:
master_df = [account_dim.columns] + [fraud_claim_case.columns] + [fraud_claim_tran.columns] + [rams_batch_cur.columns] + [statement_fact.columns] + [syf_id.columns] + [transfaction_fact.columns] + [wrld_stor_tran_fact.columns]

In [29]:
master_df

[Index(['current_account_nbr', 'client_id', 'open_date', 'card_activation_date',
        'card_activation_flag', 'ebill_ind', 'date_in_collection',
        'overlimit_type_flag', 'payment_hist_1_12_mths',
        'payment_hist_13_24_mths', 'employee_code',
        'external_status_reason_code', 'special_finance_charge_ind', 'pscc_ind',
        'ext_status_reason_cd_desc', 'account_card_type'],
       dtype='object'),
 Index(['current_account_nbr', 'case_id', 'reported_date', 'open_date',
        'close_date', 'reopen_date', 'gross_fraud_amt', 'net_fraud_amt'],
       dtype='object'),
 Index(['current_account_nbr', 'case_id', 'transaction_dt', 'transaction_am'], dtype='object'),
 Index(['cu_bhv_scr', 'ca_cash_bal_pct_crd_line', 'ca_cash_bal_pct_cash_line',
        'cu_nbr_days_dlq', 'cu_nbr_of_plastics', 'ca_avg_utilz_lst_6_mnths',
        'cu_cash_line_am', 'cu_crd_bureau_scr', 'cu_crd_line',
        'cu_next_crd_line_rev_date', 'cu_cur_balance', 'cu_cur_nbr_due',
        'ca_current_u

In [18]:
account_dim['payment_hist_1_12_mths'].unique()

array(['ZZZZZZIAZZZZ', 'IQIIIQIA', 'IIIQIQIIIIA', ..., 'ZZZA%%%-IQQQ',
       'I21IIQIIIQII', 'Z0ZZZZIIAIQQ'], shape=(5570,), dtype=object)

In [13]:
rams_batch_cur['ca_nsf_count_lst_12_months'].unique()

array([0, 1, 2, 3, 4, 5, 7])

In [5]:
fraud_claim_case[fraud_claim_case['case_id'] == 3202508301390]

,current_account_nbr,case_id,reported_date,open_date,close_date,reopen_date,gross_fraud_amt,net_fraud_amt
54,KQQdmrooVFs1Devc,3202508301390,2025-03-23,2025-03-24,NaN,NaN,373.56,373.56


In [6]:
fraud_claim_tran[fraud_claim_tran['case_id'] == 3202508301390]

,current_account_nbr,case_id,transaction_dt,transaction_am
0,KQQdmrooVFs1Devc,3202508301390,2025-03-12 20:00:00,5.29
26,KQQdmrooVFs1Devc,3202508301390,2025-03-12 20:00:00,20.00
130,KQQdmrooVFs1Devc,3202508301390,2025-03-12 20:00:00,329.98
200,KQQdmrooVFs1Devc,3202508301390,2025-03-12 20:00:00,18.29
